In [1]:
import time,requests,json,random
import pandas as pd
import datetime as dt


In [2]:
class StockCrawling:
    req = None
    db = None

    def __init__(self):
        self.req = requests.session()

    def __del__(self):
        self.req.close()

    def show_realtime(self, *stock_id):
        twse_url = 'http://mis.twse.com.tw/stock/api/getStockInfo.jsp'
        timestamp = int(time.time() * 1000)
        channels = '|'.join('tse_{}.tw'.format(target_tse) for target_tse in stock_id)
        query_url = '{}?&ex_ch={}&json=1&delay=0&_={}'.format(twse_url, channels, timestamp)

        headers = {'Accept-Language': 'zh-TW',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36',
                   }

        self.req.get('http://mis.twse.com.tw/stock/index.jsp', headers=headers)
        response = self.req.get(query_url)
        if response.text.strip() == '':
            return None

        # d: 日期, h: 最高, l: 最低, c:代號, n: 名稱, t:時間, o: 開盤, v: 交易量, z: 成交價
        # load JSON from TSE
        content = json.loads(response.text)
        self.req.cookies.clear()
        return content['msgArray']

In [13]:
if __name__ == '__main__':
    # To ill stock IDs that you want.
    data = pd.read_csv("./ListedStockID/化學類.csv")
    stock_ids = [ str(i) for i in data['代碼'].to_list() ]
    crawling = StockCrawling()
    try:
        while True:
            now = dt.datetime.now()
            #if now < STOP_TIME:
            data = crawling.show_realtime(*stock_ids)
            #print(data)
            records = []
            items = []
            for s_dict in data:
                try:
                    _t = dt.datetime.strptime(s_dict['t'], '%H:%M:%S').time().strftime('%H:%M:%S')
                    _dt = dt.datetime.strptime(s_dict['d'], "%Y%m%d").date().strftime('%Y-%m-%d')
                    #_dt: 日期；_t: 時間；c: 股票代碼；n: 股票簡稱；z: 當盤成交價；tv: 當盤成交量；
                    # h: 最高價；l: 最低價；y: 昨收；o: 開盤價；v: 累積成交量
                    if(s_dict['z']!='-'):
                        s_dict['z'] = float(s_dict['z'])
                    else:
                        s_dict['z'] = float('NaN')
                    if(s_dict['tv']!='-'):
                        s_dict['tv'] = float(s_dict['tv'])
                    else:
                        s_dict['tv'] = float('NaN')
                    if(s_dict['h']!='-'):
                        s_dict['h'] = float(s_dict['h'])
                    else:
                        s_dict['h'] = float('NaN')
                    if(s_dict['l']!='-'):
                        s_dict['l'] = float(s_dict['l'])
                    else:
                        s_dict['l'] = float('NaN')
                    if(s_dict['o']!='-'):
                        s_dict['o'] = float(s_dict['o'])
                    else:
                        s_dict['o'] = float('NaN')
                    if(s_dict['v']!='-'):
                        s_dict['v'] = float(s_dict['v'])
                    else:
                        s_dict['v'] = float('NaN')

                    records.append([_dt, _t, s_dict['c'], s_dict['n'], s_dict['z'], s_dict['tv'],
                                    s_dict['h'], s_dict['l'], float(s_dict['y']), float(s_dict['o']), int(s_dict['v'])])

                except ValueError as _e:
                    print("ValueErr : " + str(_e))
            print(records)
            _sleep_time = random.randint(2, 4)
                # sleep
            time.sleep(_sleep_time)
            print("====== " + time.strftime("%H:%M:%S") + " ======")
    finally:
        del crawling


[['2022-08-18', '13:17:48', '1701', '中化', 21.65, 2.0, 21.85, 21.55, 21.7, 21.65, 444], ['2022-08-18', '13:17:18', '1702', '南僑', 45.55, 1.0, 45.95, 45.45, 45.65, 45.55, 140], ['2022-08-18', '13:17:36', '1707', '葡萄王', nan, nan, 137.0, 134.5, 134.5, 134.5, 277], ['2022-08-18', '13:17:50', '1708', '東鹼', nan, nan, 49.1, 48.25, 49.05, 48.9, 5246], ['2022-08-18', '13:17:42', '1709', '和益', nan, nan, 26.15, 25.9, 25.9, 26.0, 1267], ['2022-08-18', '13:17:50', '1710', '東聯', nan, nan, 17.85, 17.5, 17.7, 17.7, 931], ['2022-08-18', '13:17:48', '1711', '永光', nan, nan, 20.95, 20.45, 20.85, 20.8, 1522], ['2022-08-18', '13:17:48', '1712', '興農', nan, nan, 38.5, 37.1, 38.6, 38.3, 3451], ['2022-08-18', '12:30:16', '1713', '國化', 24.3, 1.0, 24.3, 24.25, 24.3, 24.25, 8], ['2022-08-18', '13:16:34', '1714', '和桐', nan, nan, 9.32, 9.29, 9.3, 9.31, 482], ['2022-08-18', '13:17:00', '1717', '長興', nan, nan, 32.95, 32.6, 33.0, 32.95, 1034], ['2022-08-18', '13:17:30', '1718', '中纖', nan, nan, 8.14, 8.06, 8.14, 8.14, 133

KeyboardInterrupt: 